# Model Development

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.linear_model import Lasso
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV, LinearRegression, ElasticNet,  HuberRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import time

ModuleNotFoundError: No module named 'pandas'

## 1.0 Data
The data being loaded below has already been processed in the data processing notebook of this project.

In [ ]:
data = pd.read_csv('input/processed_data_nyc.csv', index_col = 0)
numerical_data = pd.read_csv('input/processed_data_nyc_numerical.csv')
data.head()

Since we are predicting price in this notebook, we will take price as the label (y vector) and we will drop it from the features.

In [ ]:
y = data.price
data = data.drop(['price'], axis=1)

Now we turn the data and label to numpy arrays, which is more convenient for computations.

In [ ]:
X = np.asarray(data)
y = np.asarray(y).ravel()

To test our model, we need to split the data into training and testing sets.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("Training Dataset: {}".format(X_train.shape))
print("Testing Dataset: {}".format(X_test.shape))

For better results, data should be scaled to a zero mean and unit variance. This can be done using the robust scaler from scikit-learn.

In [ ]:
scaler = RobustScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

## 2.0 Models
Before we begin testing different models, we will declare a general function for cross-validation. I will use k-fold validation with 5 folds evaluated based on the mean squared error.

In [ ]:
n_folds = 5

def rmse_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state = 91).get_n_splits(numerical_data)
    return cross_val_score(model, X_train, y_train, scoring='neg_mean_squared_error', cv=kf)

Now let's do a basic cross-validation test on some well-known regression models without any fine-tuning. We want to get an idea of what's the best model to invest our time in!

In [ ]:
MODELS = [LinearRegression, Ridge, Lasso, ElasticNet, RandomForestRegressor, HuberRegressor]

for Model in MODELS:
    cv_res = rmse_cv(Model())
    print('{}: {:.5f} +/- {:5f}'.format(Model.__name__, -cv_res.mean(), cv_res.std()))

Looking at the above results, the three seemingly best classifiers for this application are:  
1) Random Forest Regressor  
2) Ridge Regressor   
3) Huber Regressor   

As such, I will test these three models and tune their hyperparameters to see which can achieve the best results

### 2.1 Random Forest Regression
In Random Forest Regression, the main parameter to be hypertuned is the number of estimators, which is the number of decision trees used in this ensemble model.

In [ ]:
num_estimators = [2, 5, 10, 20, 50, 100, 200, 300]

cv_randomForest = []
cv_randomForest_time = []
counter = 0
for n in num_estimators:
    time1 = time.time()
    cv_temp = -rmse_cv(RandomForestRegressor(n_estimators = n)).mean()
    cv_randomForest.append(cv_temp)
    time2 = time.time()
    timeF = time2- time1
    print("Number of estimators {}: MSE = {}, Time = {}".format(n, cv_randomForest[counter], timeF))
    cv_randomForest_time.append(timeF)
    counter+=1
    

Now we can plot the cross-validated mean squared error (MSE) for each number of estimators tested.

In [ ]:
cv_randomForest = pd.Series(cv_randomForest, index = num_estimators)
cv_randomForest_time = pd.Series(cv_randomForest_time, index = num_estimators)

fig, axes = plt.subplots(1,2,figsize=(21, 8))

cv_randomForest.plot(title = "Random Forest RMSE", style = '-+', ax = axes[0])
axes[0].set_xlabel("Number of Estimators") 
axes[0].set_ylabel("RMSE")

cv_randomForest_time.plot(title = "Random Forest Training Time", style = '-+', ax = axes[1])
axes[1].set_xlabel("Number of Estimators") 
axes[1].set_ylabel("Time (seconds)")

plt.savefig('output/figures/random_forest_cv')

Looking at the plot above, we can see that increasing the number of estimators improves the performance of the model. However, this comes at the cost of increased computational burden. Higher number of estimators also means longer training time. So we need to pick a good trade off point where we can achieve computational efficiency. The change in MSE between 50 and 300 is very minor but the change in computational burden (measured through training time) is quite large. For that reason, the number of estimators chosen will be 50.

### 2.2 Ridge Regression
Ridge regression has two main parameters to tune. The first is the solver used to train the regression model, and the regularization strength, alpha, which reduces the variance of the estimates. In the scikit-learn library we are using, the solver is set to auto by default. This means that the algorithm will choose the solver based on the nature of the data automatically. As such, we will only cross validate for alpha values.

In [ ]:
alphas = [0.001, 0.01, 0.1, 0.3, 0.5, 1, 1.5, 2, 5, 10, 20, 50, 100]

cv_ridge = []
cv_ridge_time = []
counter = 0
for a in alphas:
    time1 = time.time()
    cv_temp = -rmse_cv(Ridge(alpha = a)).mean()
    cv_ridge.append(cv_temp)
    time2 = time.time()
    timeF = time2- time1
    print("Alpha {}: MSE = {}, Time = {}".format(a, cv_ridge[counter], timeF))
    cv_ridge_time.append(timeF)
    counter+=1

In [ ]:
cv_ridge = pd.Series(cv_ridge, index = alphas)
cv_ridge_time = pd.Series(cv_ridge_time, index = alphas)

fig, axes = plt.subplots(1,2,figsize=(21, 8))

cv_ridge.plot(title = "Ridge RMSE", style = '-+', ax = axes[0])
axes[0].set_xlabel("Alpha") 
axes[0].set_ylabel("RMSE")

cv_ridge_time.plot(title = "Ridge Training Time", style = '-+', ax = axes[1])
axes[1].set_xlabel("Alpha") 
axes[1].set_ylabel("Time (seconds)")

plt.savefig('output/figures/ridge_cv')

As can be seen above, changing the parameters of the ridge regressor had negligible impact on the performance. We will choose alpha of 5 due to best performance with no compromise on training time.

### 2.3 Huber Regression
Huber Regressors have two primary parameters, epsilon and alpha. Alpha is the regularization parameter and epsilon controls the number of samples that should be classified as outliers. The smaller the epsilon, the more robust it is to outliers. Epsilon is a value greater than 1. We will test for both of these parameters using gridsearch.

In [ ]:
parameters = {'alpha':[0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100], 
              'epsilon':[1, 1.1, 1.2,1.25, 1.3 , 1.35, 1.5, 1.75, 2, 2.5, 3]}
huberModel = HuberRegressor()

clf = GridSearchCV(huberModel, parameters, verbose=5, scoring='neg_mean_squared_error')
clf.fit(X_train, y_train)

In [ ]:
print(-clf.cv_results_['mean_test_score'])

By looking at the mean test scores above, we can clearly see that there is very little difference in performance from changing the hyper-parameters. Now, let's look at the difference in training time:

In [ ]:
print(clf.cv_results_['mean_fit_time'])

Since the difference in training time and score are both negligible, we will pick the parameters with the highest score in this gridsearch:

In [ ]:
print(clf.best_params_)

## 3.0 Final Tests
In this notebook, we explored the most popular regression models, and then tested and tuned three different methods. The scoring method used the mean squared error. The final parameters chosen for each model are:  

1) Random Forest: Number of estimators = 50  
2) Ridge Regression: Alpha = 5  
3) Huber Regression: Alpha = 10, Epsilon = 3  

As such, we will define our final models:

In [ ]:
randomForest_final = RandomForestRegressor(n_estimators=50)
ridge_final = Ridge(alpha=5)
huber_final = HuberRegressor(alpha=10, epsilon=3)

Now we train each of those models on the entire training set, and test it on the test set:

In [ ]:
# Random Forest
randomForest_final.fit(X_train,y_train)
rf_y = randomForest_final.predict(X_test)
rf_result = mean_squared_error(y_test, rf_y)
print("Random Forest: {}".format(rf_result))

# Ridge
ridge_final.fit(X_train,y_train)
ridge_y = ridge_final.predict(X_test)
ridge_result = mean_squared_error(y_test, ridge_y)

print("Ridge : {}".format(ridge_result))

# Huber
huber_final.fit(X_train,y_train)
huber_y = huber_final.predict(X_test)
huber_result = mean_squared_error(y_test,huber_y)
print("Huber: {}".format(huber_result))

## 4.0 Saving Results
Now we will save the data for future use so we do not have to run this notebook again.

In [ ]:
models = ['Random Forest', 'Ridge', 'Huber']
parameters = ['N_Estimators = 50', 'Alpha = 5', 'Alpha = 10, Epsilon = 3']
final_mse = [rf_result, ridge_result, huber_result]

finalResult_df = pd.DataFrame({
    'Model': models,
    'Parameters': parameters,
    'MSE': final_mse,
})

finalResult_df.head()

In [ ]:
randomForest_df = pd.concat([cv_randomForest, cv_randomForest_time], axis=1)
randomForest_df.columns = ['MSE', 'Time']

ridge_df = pd.concat([cv_ridge, cv_ridge_time], axis=1)
ridge_df.columns = ['MSE', 'Time']

huber_df = pd.DataFrame(clf.cv_results_)

In [ ]:
writer = pd.ExcelWriter('output/results_cv.xlsx', engine='xlsxwriter')
randomForest_df.to_excel(writer, sheet_name='Random Forest')
ridge_df.to_excel(writer, sheet_name='Ridge')
huber_df.to_excel(writer, sheet_name='Huber')
finalResult_df.to_excel(writer, sheet_name='Final Models Results')
writer.save()